In [1]:
from Bio.codonalign.codonseq import CodonSeq
from Bio.codonalign.codonseq import cal_dn_ds
import random
from comp_dnds import dnds
from comp_dnds.paths import hamming_distance
import time
import pandas as pd
from plotnine import *
import numpy as np
from tqdm.autonotebook import tqdm
from interruptingcow import timeout

/Users/maxime/Documents/github/comp_dnds/.venv/lib/python3.9/site-packages/Bio/codonalign/__init__.py:23: BiopythonExperimentalWarning: Bio.codonalign is an experimental module which may undergo significant changes prior to its future official release.
  warnings.warn(
/var/folders/1c/l1qb09f15jddsh65f6xv1n_r0000gp/T/ipykernel_48672/2819809054.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
from Bio import __version__ as biopython_version
biopython_version

'1.81'

In [3]:
def biopython_dnds(ref_seq, obs_seq):
    ref_codons = CodonSeq(ref_seq)
    obs_codons = CodonSeq(obs_seq)
    return cal_dn_ds(ref_codons, obs_codons, method='NG86')

In [4]:
d = dnds()

In [5]:
help(d)

Help on dnds in module comp_dnds.dnds object:

class dnds(builtins.object)
 |  dnds(alphabet: List[str] = ['A', 'T', 'G', 'C'], codon_size: int = 3, genetic_code: int = 1) -> None
 |  
 |  Compute dn and ds for two sequences using the Nei-Gojobori(1986) method.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, alphabet: List[str] = ['A', 'T', 'G', 'C'], codon_size: int = 3, genetic_code: int = 1) -> None
 |      Constructor for the dnds class.
 |      
 |      Args:
 |          alphabet (List[str], optional): Letters of DNA alphabet. Defaults to ['A','T','G','C'].
 |          codon_size (int, optional): Number of DNA base pairs in a codon. Defaults to 3.
 |          genetic_code (int, optional): Identifier of the genetic code.
 |              See https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi for reference.
 |              Defaults to 1.
 |  
 |  compute(self, ref_seq: str, obs_seq: str, bootstrap: int = 0, cpus: int = 8) -> Tuple[float, float]
 |      Compute dn and ds 

In [6]:
d = dnds()
def run_comp_dnds(ref_seq, obs_seq):
    return d.compute(ref_seq, obs_seq)

In [7]:
def seq_gen(length):
    return ''.join(random.choice('ATGC') for _ in range(length))

In [8]:
def mutate_sequence(seq, prob_mutation):
    return ''.join(random.choice('ATGC') if random.random() < prob_mutation else seq[i] for i in range(len(seq)))

In [9]:
reference = [seq_gen(999) for i in range(1000)]
observed = [mutate_sequence(seq, 0.01) for seq in reference]

Execution time benchmark

In [10]:
biopython_durations = []
comp_dnds_durations = []
for r, o in zip(reference, observed):
    # biopython cal_dn_ds
    start = time.time()
    biopython_dnds(r, o)
    end = time.time()
    duration = end - start
    # comp_dnds
    start = time.time()
    run_comp_dnds(r, o)
    end = time.time()
    duration2 = end - start
    biopython_durations.append(duration)
    comp_dnds_durations.append(duration2)

In [11]:
df = pd.DataFrame({'biopython': biopython_durations, 'comp_dnds': comp_dnds_durations})
df_melt = df.melt(value_name='duration', var_name='method')


In [12]:
df_stats = df_melt.groupby('method').describe().reset_index()

In [13]:
df_stats.columns = df_stats.columns.droplevel(0)

In [14]:
df_stats

,,count,mean,std,min,25%,50%,75%,max
0,biopython,1000.0,0.019489,0.002643,0.018637,0.019100,0.019284,0.019544,0.100843
1,comp_dnds,1000.0,0.000617,0.000046,0.000583,0.000597,0.000602,0.000611,0.001191


In [15]:
df_stats.columns = ['method'] + df_stats.columns[1:].to_list()

In [16]:
df_stats

,method,count,mean,std,min,25%,50%,75%,max
0,biopython,1000.0,0.019489,0.002643,0.018637,0.019100,0.019284,0.019544,0.100843
1,comp_dnds,1000.0,0.000617,0.000046,0.000583,0.000597,0.000602,0.000611,0.001191


In [17]:
g = (
    ggplot(df_stats, aes(x='method', y='mean', fill='method')) 
    + geom_bar(stat='identity') 
    + geom_errorbar(aes(ymin="mean-std",ymax="mean+std"))
    + theme_classic()
    + labs(y='mean duration (s)', x='method')
)
g.save("../plots/biopython_benchmark.png", dpi=72)

/Users/maxime/Documents/github/comp_dnds/.venv/lib/python3.9/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/maxime/Documents/github/comp_dnds/.venv/lib/python3.9/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../plots/biopython_benchmark.png


Calculating speedup

In [18]:
(df.biopython / df.comp_dnds).describe()

count    1000.000000
mean       31.713436
std         4.608113
min        16.112913
25%        31.395310
50%        31.970225
75%        32.437253
max       164.004265
dtype: float64

Checking results consistency with biopython calc_dnds

In [19]:
agree = []
for r, o in zip(reference, observed):
    # biopython cal_dn_ds
    start = time.time()
    biodn, biods = biopython_dnds(r, o)
    # comp_dnds
    comp_dn, comp_ds, z, p = run_comp_dnds(r, o)
    agree.append(abs(biodn - comp_dn) < 1e-3 and abs(biods - comp_ds) < 1e-3)
    

In [20]:
agree.count(True) / len(agree)

0.994

The calculated dN and dS values are identical to those calculated by BioPython (difference of less than 10e3 for 99.4% of the results).

## comp_dnds bootstrap 

Testing the bootstrap

In [21]:
genetic_code_no_stop = {c:d.codon_table[c] for c in d.codon_table if d.codon_table[c] != 'stop'}

Let's mutate the reference sequence in a cleverer way:
- For each codon, we do a 1 basepair non-synonymous with a probability of `prob_ns`
- If not mutated, we do a 1 basepair non-synonymous with a probability of `prob_s`
- Otherwise, the codon remains identical

In [70]:
ref_codons = random.choices(list(genetic_code_no_stop.keys()), k=15)
def mutate(ref_codons, prob_ns, prob_s):
    s = 0
    ns = 0
    codons = []
    for c in tqdm(ref_codons):
        mutated = False
        if random.random() < prob_ns:
            same = True
            while same == True:
                new_c = random.choice(list(genetic_code_no_stop.keys()))
                d = hamming_distance(c, new_c)
                if genetic_code_no_stop[new_c] != genetic_code_no_stop[c] and d == 1:
                    codons.append(new_c)
                    same = False
                    mutated = True
                    ns += d
                    continue
        elif random.random() < prob_s:
            same = False
            while same == False:
                new_c = random.choice(list(genetic_code_no_stop.keys()))
                d = hamming_distance(c, new_c)
                if genetic_code_no_stop[new_c] == genetic_code_no_stop[c] and d == 1:
                    codons.append(new_c)
                    same = True
                    mutated = True
                    s += d
                    continue
        if mutated == False:
            codons.append(c)
    print(f"non-synonymous mutations in codons ({ns})")
    print(f"synonymous mutations in codons ({s})")
    return codons
try:
    with timeout(0.5, exception=RuntimeError):
        mutated_codons = mutate(ref_codons, prob_ns=0.7, prob_s=0.1)
        dn, ds, z , p = d.compute("".join(ref_codons), "".join(mutated_codons))
except RuntimeError:
    pass
# ref_codons = ["CCC"]*100
# mutated_codons = ["CCC"]*49 + ["CCA"] + ["CGC"]*50


100%|██████████| 15/15 [00:00<00:00, 20106.92it/s]

non-synonymous mutations in codons (13)
synonymous mutations in codons (1)


In [71]:
print(dn/ds)

6.9053189758991165


In [79]:
dn, ds, z, p = d.compute("".join(ref_codons), "".join(mutated_codons), bootstrap=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 12543.26it/s]


In [80]:
print(f"ω= {round(dn/ds,2)} - z-score= {round(z,2)} - p-val= {round(p, 5)}")

ω= 6.91 - z-score= 3.72 - p-val= 0.0002


In [81]:
print(" ".join(ref_codons))
print(" ".join(mutated_codons))

GCC GGG GGA AGG ACA TAT CTC GCT CCA CCT AAT GGA ATC ATC GGT
GAC GGC CGA GGG GCA AAT CGC ACT ACA ACT ACT GAA GTC ATC AGT
